In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import utils

In [2]:
class ReconstructingRegressor(keras.models.Model):
    def __init__(self, output_dim, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.hidden = [keras.layers.Dense(30, activation='relu') for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)

    def build(self, input_shape):
        num_inputs = input_shape[-1]
        self.reconstruct = keras.layers.Dense(num_inputs)
        return super().build(input_shape)

    def call(self, inputs, training=None, mask=None):
        a = inputs
        for layer in self.hidden:
            a = layer(a)
        reconstruction = self.reconstruct(a)
        self.reconstruction_loss = 0.05 * tf.reduce_mean(tf.square(reconstruction - inputs))
        return self.out(a)

    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x)
            loss = self.compiled_loss(y, y_pred, regularization_losses=[self.reconstruction_loss])

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return {m.name: m.result() for m in self.metrics}

In [3]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = utils.load_housing_data()
print(f"x_train.shape = {x_train.shape}, y_train.shape = {y_train.shape}")
print(f"x_val.shape = {x_val.shape}, y_val.shape = {y_val.shape}")
print(f"x_test.shape = {x_test.shape}, y_test.shape = {y_test.shape}")

x_train.shape = (11610, 8), y_train.shape = (11610,)
x_val.shape = (3870, 8), y_val.shape = (3870,)
x_test.shape = (5160, 8), y_test.shape = (5160,)


In [4]:
utils.reset_session()

model = ReconstructingRegressor(1)
model.compile(loss='mse', optimizer="nadam")
history = model.fit(x_train, y_train, epochs=5)

Epoch 1/5
363/363 [==============================] - 0s 672us/step - loss: 0.9017
Epoch 2/5
363/363 [==============================] - 0s 653us/step - loss: 0.4123
Epoch 3/5
363/363 [==============================] - 0s 643us/step - loss: 0.3835
Epoch 4/5
363/363 [==============================] - 0s 634us/step - loss: 0.3559
Epoch 5/5
363/363 [==============================] - 0s 625us/step - loss: 0.3403
